# 多输入与多输出通道
如：彩色图像具有RGB三个通道。

当我们添加通道时，我们的输入和隐藏的表示都变成了三维张量。例如，每个RGB输入图像具有$3 \times h \times w$的形状。

## 1. 多输入通道
对每个通道执行互相关操作，然后对每个通道互相关操作产生的结果对应位置相加

In [33]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 遍历所有通道，然后再加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [34]:
'''验证操作'''
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 2. 多输出通道

在网络架构中，随着网络层数加深，我们常会增加输出通道维数，通过减少空间分辨率以获得更大的通道深度。我们可以将每个通道看作对不同特征的响应。因为每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道检测器。

使用$c_i$和$c_o$分别表示输入和输出通道的数目，并让$k_h$和$k_w$为卷积核的高度和宽度。为了获得多个通道输出，我们可以为每个输出通道创建一个形状为$c_i \times k_h \times k_w$的卷积核张量，这样的卷积核的形状为$c_o \times c_i \times k_h \times k_w$。在互相关运算中，每个输出通道先获取所有输入通道，再以对应该输出通道的卷积核计算出结果

In [35]:
'''实现计算多个通道的输出互相关函数'''

def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], dim=0)

In [36]:
'''通过将核张量K与K+1和K+2链接起来，构造一个足有3个输出通道的卷积核'''
K = torch.stack((K, K+1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [37]:
'''对输入张量X与卷积核张量K执行互相关运算。现在的输出包含三个通道，第一个通道的结果与先前输入张量X和多输入单输出通道的结果一致'''
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 3. 1*1卷积核
由于使用了最小窗口，失去了卷积层的特有能力，在高度和宽度维度上，识别相邻元素间相互作用的能力。1*1卷积的唯一计算发生在通道上

更加详细的解释请查阅 https://zh.d2l.ai/chapter_convolutional-neural-networks/channels.html


In [38]:
'''使用全连接层实现1 * 1卷积， 需要对输入和输入数据形状进行调整'''
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [39]:
'''验证功能'''
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6
